In [1]:
import socket
import ssl
import time
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key():
    with open("private_key.pem", "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key

def sign_message(private_key, message):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def send_secure_message():
    message = b"This is a test message from the cardholder."
    private_key = load_private_key()
    signature = sign_message(private_key, message)

    # 创建SSL上下文
    context = ssl.create_default_context()
    context.load_verify_locations('server.crt')  # 加载服务器证书位置

    # 创建安全套接字
    with socket.create_connection(('localhost', 65432)) as sock:
        with context.wrap_socket(sock, server_hostname='localhost') as ssock:
            ssock.sendall(message + signature)  # 发送消息和签名
            print("Message and signature sent securely!")
            # 增加延时以等待服务器处理
            time.sleep(2)

if __name__ == "__main__":
    send_secure_message()


Message and signature sent securely!
